In [16]:
import time
import dlib
import csv
import os
import features
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import cPickle as pickle
import itertools as it
import more_itertools as mit
from collections import namedtuple

import multiprocessing
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [25]:
TRAINING_FOLDER = '/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-nov-6/'
OUTPUT_CSV_FILE = 'gaze_vectors_with_failures_2.csv'

# Gaze vector and failure extraction

In [21]:
# Each image will have its gaze vector extracted for each face detected in the image.
# Failures will also be reported (face_detection_error, left/right_pupil_error)
#
# Row Format:
# file_path,failure1:failure2:failure3,x1,x2,...

def dets_in_rois_generator(rois, frame):
    for roi in rois:
        y1 = roi.top() - 100
        y2 = roi.bottom() + 100
        x1 = roi.left() - 100
        x2 = roi.right() + 100
        for det in features.FaceDetection.calculate(frame[y1:y2, x1:x2].copy()):
            yield det

def file_to_gaze_vector_with_erorrs_csv_rows(file_path, rois=[]):
    frame = skimage.io.imread(file_path)
    
    if not rois:
        dets = features.FaceDetection.calculate(frame)
        if not dets:
            return ([], [(file_path, 'face_detection_error')])
    else:
        dets = list(dets_in_rois_generator(rois, frame))
        if not dets:
            return file_to_gaze_vector_with_erorrs_csv_rows(file_path, [])
    
    rows = []
    
    for det in dets:
        failures = []

        landmarks = features.FaceLandmarks.calculate(frame, det)
        orientation = features.HeadOrientation.calculate(frame, landmarks)
        eye_roi = features.EyeRegions.calculate(landmarks)
        left_pupil = features.IrisFinder.calculate(frame, eye_roi.left)
        right_pupil = features.IrisFinder.calculate(frame, eye_roi.right)
        ellipses = features.EyeEllipses.calculate(landmarks)
        
        if left_pupil == ((0,0), 0):
            failures.append('left_pupil_error')
        if right_pupil == ((0,0), 0):
            failures.append('right_pupil_error')

        vector = features.GazeMatrix.calculate(
            landmarks,
            orientation,
            ellipses,
            left_pupil,
            right_pupil
        )

        rows.append([file_path, ':'.join(failures)] + vector)
        
    # file_path,failure1:failure2:failure3,x1,x2,...
    return (dets, rows)

## Build file list

In [5]:
rejected_dirs = (
#     'back_left', 'back_right', 'back_center',
#     'front_center', 'front_left', 'front_right',
#     'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18',
#     'p19', 'p20', 'p21', 'p22', 'p23', 'p9', 'p8', 'p7', 'p6', 'p5',
#     'p4', 'p3', 'p2', 'p2-glasses'
)
training_files = []
for root, dirs, files in os.walk(TRAINING_FOLDER):
    filtered_files = (f for f in files if f != '.DS_Store')
    for file_ in filtered_files:
        training_files.append(os.path.join(root, file_))
        
    for rd in rejected_dirs:
        if rd in dirs:
            dirs.remove(rd)

## Process files and extract feature and failures

**Note: The following process takes a while.**

In [22]:
print "Time started:", time.time()

out_file = open(OUTPUT_CSV_FILE, 'w')
writer = csv.writer(out_file)
tracker = dlib.correlation_tracker()
rois = []
for file_path in training_files:
    (updated_rois, rows) = file_to_gaze_vector_with_erorrs_csv_rows(file_path, rois)
    writer.writerows(rows)
    rois = updated_rois
    out_file.flush()

print "Time ended:", time.time()

Time started: 1447186912.99
Time ended: 1447188889.31


# Dataset Analysis

In [23]:
Sample = namedtuple('Sample', ['path','label','vector'])

def map_file_path_to_number(file_path):
    for label in LABELS:
        if label in file_path:
            return labels_to_number[label]
    return -1

def map_row_to_sample(csv_row):
    return Sample(
        path=row[0],
        label=map_file_path_to_number(row[0]),
        vector=[float(x) for x in row[2:]]
    )

def failures(rows):
    for row in rows:
        if row[1]:
            for failure in row[1].split(':'):
                yield failure

vectors_dump_file = open(OUTPUT_CSV_FILE, 'rb')
vectors_and_failures = list(csv.reader(vectors_dump_file))

LABELS = ['top_left', 'top_right', 'bot_left', 'bot_right']
labels_to_number = dict(zip(LABELS, range(len(LABELS))))
samples = [map_row_to_sample(row) for row in vectors_and_failures if not row[1]]

paths = [s.path for s in samples]
detections_per_file_counts = [[x,paths.count(x)] for x in set(paths)]
multiple_detections = [x for x in detections_per_file_counts if x[1] > 1]


print "Total number of files:\t", len(training_files)
print "Total vectors:\t\t", len(vectors_and_failures)
print "Complete vectors:\t", len(samples) 
print ""
print "Left Pupil errors:\t", sum((1 for f in failures(vectors_and_failures) if f == 'left_pupil_error'))
print "Right Pupil errors:\t", sum((1 for f in failures(vectors_and_failures) if f == 'right_pupil_error'))
print "Face detection errors:\t", sum((1 for f in failures(vectors_and_failures) if f == 'face_detection_error'))
print ""
print "Numer of multiple detections: ", len(multiple_detections)

Total number of files:	4928
Total vectors:		5061
Complete vectors:	3796

Left Pupil errors:	18
Right Pupil errors:	23
Face detection errors:	1238

Numer of multiple detections:  115
